# Numpy Implementation of RNN

In [20]:
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

with open('data/reddit-comments-2015-08.csv', 'rb' ) as f:
    reader = csv.reader(f, skipinitialspace = True)
    reader.next()
    # split doc into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(doc[0].decode('utf-8').lower()) for doc in reader])
    sentences = ["%s %s %s" % (sentence_start_token, sent, sentence_end_token) for sent in sentences]
    print "Parsed {} sentences.".format(len(sentences))

Parsed 79170 sentences.
Parsed 79170 sentences.


In [22]:
token_sentences = [nltk.word_tokenize(sent) for sent in sentences]
token_freq = nltk.FreqDist(itertools.chain(*token_sentences))
print "Total of {} unique words tokens".format(len(token_freq))

Total of 65751 unique words tokens
Total of 65751 unique words tokens


In [23]:
vocab = token_freq.most_common(vocabulary_size - 1)
index2vocab = [word[0] for word in vocab]
index2vocab.append(unknown_token)
word2index = {word: i for i, word in enumerate(index2vocab)}
print "With a vocabulary sized restricted to {}".format(len(word2index))
print "The least frequent word is '{}' with {} counts in the reddit comments".format(vocab[-1][0],vocab[-1][1])

With a vocabulary sized restricted to 8000
The least frequent word is 'devoted' with 10 counts in the reddit comments
With a vocabulary sized restricted to 8000
The least frequent word is 'devoted' with 10 counts in the reddit comments


In [24]:
# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(token_sentences):
    token_sentences[i] = [word if word in word2index else unknown_token for word in sent]

print "\nExample sentence: '{}'".format(sentences[4])
print "\nExample sentence after Pre-processing: '{}'".format(token_sentences[4])


Example sentence: 'SENTENCE_START i put in the rules at a ranking site and noticed that top qbs had 300 points more than the top rb/wr. SENTENCE_END'

Example sentence after Pre-processing: '[u'SENTENCE_START', u'i', u'put', u'in', u'the', u'rules', u'at', u'a', u'ranking', u'site', u'and', u'noticed', u'that', u'top', u'qbs', u'had', u'300', u'points', u'more', u'than', u'the', u'top', 'UNKNOWN_TOKEN', u'.', u'SENTENCE_END']'

Example sentence: 'SENTENCE_START i put in the rules at a ranking site and noticed that top qbs had 300 points more than the top rb/wr. SENTENCE_END'

Example sentence after Pre-processing: '[u'SENTENCE_START', u'i', u'put', u'in', u'the', u'rules', u'at', u'a', u'ranking', u'site', u'and', u'noticed', u'that', u'top', u'qbs', u'had', u'300', u'points', u'more', u'than', u'the', u'top', 'UNKNOWN_TOKEN', u'.', u'SENTENCE_END']'


In [25]:
# Create the training data
X_train = np.asarray([[word2index[word] for word in sent[:-1]] for sent in token_sentences])
y_train = np.asarray([[word2index[word] for word in sent[1:]] for sent in token_sentences])

In [26]:
X_train[0:2]

array([ [0, 6, 3513, 7, 155, 794, 25, 223, 8, 32, 20, 202, 5025, 350, 91, 6, 66, 207, 5, 2],
       [0, 11, 17, 7, 3114, 6036, 7999, 7999, 6036, 2]], dtype=object)

array([ [0, 6, 3513, 7, 155, 794, 25, 223, 8, 32, 20, 202, 5025, 350, 91, 6, 66, 207, 5, 2],
       [0, 11, 17, 7, 3114, 6036, 7999, 7999, 6036, 2]], dtype=object)

In [27]:
print "x:\n{} \n{}".format(" ".join([index2vocab[index] for index in X_train[0]]), X_train[0])
print "y:\n{} \n{}".format(" ".join([index2vocab[index] for index in y_train[0]]), y_train[0])

x:
SENTENCE_START i joined a new league this year and they have different scoring rules than i 'm used to . 
[0, 6, 3513, 7, 155, 794, 25, 223, 8, 32, 20, 202, 5025, 350, 91, 6, 66, 207, 5, 2]
y:
i joined a new league this year and they have different scoring rules than i 'm used to . SENTENCE_END 
[6, 3513, 7, 155, 794, 25, 223, 8, 32, 20, 202, 5025, 350, 91, 6, 66, 207, 5, 2, 1]
x:
SENTENCE_START i joined a new league this year and they have different scoring rules than i 'm used to . 
[0, 6, 3513, 7, 155, 794, 25, 223, 8, 32, 20, 202, 5025, 350, 91, 6, 66, 207, 5, 2]
y:
i joined a new league this year and they have different scoring rules than i 'm used to . SENTENCE_END 
[6, 3513, 7, 155, 794, 25, 223, 8, 32, 20, 202, 5025, 350, 91, 6, 66, 207, 5, 2, 1]


In [82]:
class RNN:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
        
    def softmax(self, x):
        xt = np.exp(x - np.max(x))
        return xt / np.sum(xt)
        
    def forward_propagation(self, x):
        # one sentence at the time
        # total number of time steps
        T = len(x)
        s = np.zeros((T+1, self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)
        o = np.zeros((T, self.word_dim))
        for t in np.arange(T):
            s[t] = np.tanh(self.U[:, x[t]] + self.W.dot(s[t-1]))
            o[t] = self.softmax(self.V.dot(s[t]))     
        return [o, s]
    
    def predict(self, x):
        o, s = self.forward_propagation(x)
        return np.argmax(o, axis=1)
    
    def total_loss(self, x, y):
        L = 0
        # for each sentence
        for i in np.arange(len(y)):
            o, s = self.forward_propagation(x[i])
            correct_predicitons = o[np.arange(len(y[i])), y[i]]
            L += -1 * np.sum(np.log(correct_predicitons))
        return L
    
    def loss(self, x, y):
        # Divide the total loss by the number of training examples
        N = sum(len(y_i) for y_i in y)
        return self.total_loss(x, y) / N
    
    def backpropagation_tt(self, x, y):
        T = len(y)
        # forwardpropagation
        o, s = self.forward_propagation(x)
        # save the gradients in these variables
        dLdU = np.zeros(self.U.shape)
        dLdV = np.zeros(self.V.shape)
        dLdW = np.zeros(self.W.shape)
        delta_o = o
        delta_o[np.arange(len(y)), y] -= 1.
        
        # For each output backwards
        for t in np.arange(T)[::-1]:
            dLdV += np.outer(delta_o[t], s[t].T)
            delta_t = self.V.T.dot(delta_o[t]) * (1 - s[t]**2)
            
            for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
                #print "Backpropagation step t= {} bptt step= {} ".format(t, bptt_step)
                dLdW += np.outer(delta_t, s[bptt_step-1]) 
                dLdU[:, x[bptt_step]] += delta_t
                # update delta_t
                delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step - 1]**2)
        return [dLdU, dLdV, dLdW]
    
    def sgd_step(self, x, y, learning_rate):
        dLdU, dLdV, dLdW = self.backpropagation_tt(x, y)
        
        self.U -= learning_rate * dLdU
        self.V -= learning_rate * dLdV
        self.W -= learning_rate * dLdW
        
    def train_sgd(self, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
        
        losses = []
        examples_seen = 0
        
        for epoch in xrange(nepoch):
            if (epoch % evaluate_loss_after == 0):
                loss = self.loss(X_train, y_train)
                losses.append((examples_seen, loss))
                time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print "{}: Loss after num_examples_seen={} epoch={}: {}".format(time, examples_seen, epoch, loss)
                # Adjust the learning rate if loss increases
                if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                    learning_rate *= 0.5
                    print "Setting learning rate to {}".format(learning_rate)
                sys.stdout.flush()
                        
            for i in range(len(y_train)):
                self.sgd_step(X_train[i], y_train[i], learning_rate)
                examples_seen += 1
                
    
    def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
        # Calculate the gradients using backpropagation.
        bptt_gradients = model.backpropagation_tt(x, y)
        # List of all parameters we want to check.
        model_parameters = ['U', 'V', 'W']
        # Gradient check for each parameter
        for pidx, pname in enumerate(model_parameters):
            # Get the actual parameter value from the mode, e.g. model.W
            parameter = operator.attrgetter(pname)(self)
            print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
            # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
            it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index
                # Save the original value so we can reset it later
                original_value = parameter[ix]
                # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
                parameter[ix] = original_value + h
                gradplus = model.total_loss([x],[y])
                parameter[ix] = original_value - h
                gradminus = model.total_loss([x],[y])
                estimated_gradient = (gradplus - gradminus)/(2*h)
                # Reset parameter to original value
                parameter[ix] = original_value
                # The gradient for this parameter calculated using backpropagation
                backprop_gradient = bptt_gradients[pidx][ix]
                # calculate The relative error: (|x - y|/(|x| + |y|))
                relative_error = np.abs(backprop_gradient - estimated_gradient)/
                                (np.abs(backprop_gradient) + np.abs(estimated_gradient))
                # If the error is to large fail the gradient check
                if relative_error > error_threshold:
                    print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                    print "+h Loss: %f" % gradplus
                    print "-h Loss: %f" % gradminus
                    print "Estimated_gradient: %f" % estimated_gradient
                    print "Backpropagation gradient: %f" % backprop_gradient
                    print "Relative Error: %f" % relative_error
                    return 
                it.iternext()
            print "Gradient check for parameter %s passed." % (pname)
            


In [70]:
np.random.seed(10)
RNN_model = RNN(vocabulary_size)
o, s = RNN_model.forward_propagation(X_train[10])
print o.shape
print o

(45, 8000)
[[ 0.00012408  0.0001244   0.00012603 ...,  0.00012515  0.00012488
   0.00012508]
 [ 0.00012536  0.00012582  0.00012436 ...,  0.00012482  0.00012456
   0.00012451]
 [ 0.00012387  0.0001252   0.00012474 ...,  0.00012559  0.00012588
   0.00012551]
 ..., 
 [ 0.00012414  0.00012455  0.0001252  ...,  0.00012487  0.00012494
   0.0001263 ]
 [ 0.0001252   0.00012393  0.00012509 ...,  0.00012407  0.00012578
   0.00012502]
 [ 0.00012472  0.0001253   0.00012487 ...,  0.00012463  0.00012536
   0.00012665]]
(45, 8000)
[[ 0.00012408  0.0001244   0.00012603 ...,  0.00012515  0.00012488
   0.00012508]
 [ 0.00012536  0.00012582  0.00012436 ...,  0.00012482  0.00012456
   0.00012451]
 [ 0.00012387  0.0001252   0.00012474 ...,  0.00012559  0.00012588
   0.00012551]
 ..., 
 [ 0.00012414  0.00012455  0.0001252  ...,  0.00012487  0.00012494
   0.0001263 ]
 [ 0.0001252   0.00012393  0.00012509 ...,  0.00012407  0.00012578
   0.00012502]
 [ 0.00012472  0.0001253   0.00012487 ...,  0.00012463  0.000

In [71]:
pred = RNN_model.predict(X_train[10])
print pred.shape
print pred
print " ".join([index2vocab[index] for index in pred])

(45,)
[1284 5221 7653 7430 1013 3562 7366 4860 2212 6601 7299 4556 2481  238 2539
   21 6548  261 1780 2005 1810 5376 4146  477 7051 4832 4991  897 3485   21
 7291 2007 6006  760 4864 2182 6569 2800 2752 6821 4437 7021 7875 6912 3575]
students shortly museum ruining background hunt madden wr chicken immoral hadith lighter rude questions achieve but sells making fill arguing purchase grows feat head lube winners downside states steal but researchers christian utilize fire domain resolution 10-15 genuinely magical worship в branches memes node preferred
(45,)
[1284 5221 7653 7430 1013 3562 7366 4860 2212 6601 7299 4556 2481  238 2539
   21 6548  261 1780 2005 1810 5376 4146  477 7051 4832 4991  897 3485   21
 7291 2007 6006  760 4864 2182 6569 2800 2752 6821 4437 7021 7875 6912 3575]
students shortly museum ruining background hunt madden wr chicken immoral hadith lighter rude questions achieve but sells making fill arguing purchase grows feat head lube winners downside states steal but r

In [72]:
# print "Expected Loss for random predictions: ", np.log(vocabulary_size)
# print "Current Loss: ",RNN_model.loss(X_train[:1000], y_train[:1000])

In [73]:
# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 100
np.random.seed(10)
model = RNN(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

Backpropagation step t= 3 bptt step= 3 
Backpropagation step t= 3 bptt step= 2 
Backpropagation step t= 3 bptt step= 1 
Backpropagation step t= 3 bptt step= 0 
Backpropagation step t= 2 bptt step= 2 
Backpropagation step t= 2 bptt step= 1 
Backpropagation step t= 2 bptt step= 0 
Backpropagation step t= 1 bptt step= 1 
Backpropagation step t= 1 bptt step= 0 
Backpropagation step t= 0 bptt step= 0 
Performing gradient check for parameter U with size 1000.
Gradient check for parameter U passed.Backpropagation step t= 3 bptt step= 3 
Backpropagation step t= 3 bptt step= 2 
Backpropagation step t= 3 bptt step= 1 
Backpropagation step t= 3 bptt step= 0 
Backpropagation step t= 2 bptt step= 2 
Backpropagation step t= 2 bptt step= 1 
Backpropagation step t= 2 bptt step= 0 
Backpropagation step t= 1 bptt step= 1 
Backpropagation step t= 1 bptt step= 0 
Backpropagation step t= 0 bptt step= 0 
Performing gradient check for parameter U with size 1000.
Gradient check for parameter U passed.
Perform

/Users/marvinbertin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:125: RuntimeWarning: invalid value encountered in double_scalars
/Users/marvinbertin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:125: RuntimeWarning: invalid value encountered in double_scalars


In [83]:
np.random.seed(10)
# Train on a small subset of the data to see what happens
model = RNN(vocabulary_size)
losses = model.train_sgd(X_train[:100], y_train[:100], nepoch=10, evaluate_loss_after=1)

2015-11-01 23:13:56: Loss after num_examples_seen=0 epoch=0: 8.9874245324
2015-11-01 23:13:56: Loss after num_examples_seen=0 epoch=0: 8.9874245324
2015-11-01 23:14:09: Loss after num_examples_seen=100 epoch=1: 8.97627030828
2015-11-01 23:14:09: Loss after num_examples_seen=100 epoch=1: 8.97627030828
2015-11-01 23:14:23: Loss after num_examples_seen=200 epoch=2: 8.96021174257
2015-11-01 23:14:23: Loss after num_examples_seen=200 epoch=2: 8.96021174257
2015-11-01 23:14:37: Loss after num_examples_seen=300 epoch=3: 8.93042979657
2015-11-01 23:14:37: Loss after num_examples_seen=300 epoch=3: 8.93042979657
2015-11-01 23:14:49: Loss after num_examples_seen=400 epoch=4: 8.86226430002
2015-11-01 23:14:49: Loss after num_examples_seen=400 epoch=4: 8.86226430002
2015-11-01 23:15:02: Loss after num_examples_seen=500 epoch=5: 6.91356992751
2015-11-01 23:15:02: Loss after num_examples_seen=500 epoch=5: 6.91356992751
2015-11-01 23:15:14: Loss after num_examples_seen=600 epoch=6: 6.30249269658
2015-